# Advanced Machine Learning - Practicum 2 - Evaluating Clustering Algorithms

**Topics covered**: Performance measurements for un-supervised learning

**Deliverables**:
- Complete the tasks as detailed in this document.
- You are required to implement two metrics calculation and compare the result with the one produced by scikit-learn.

**Objectives**:  
Get familiarized with clustering performance evaluation.

---


In [1]:
import numpy as np                                # for your implementation
from sklearn.metrics import pairwise_distances    # pairwise_distances() or you could choose
from scipy.spatial import distance                # cdist() to computer pairs of vectors
from sklearn import metrics                       # for comparison
from sklearn.cluster import KMeans                # for comparison
from sklearn import datasets

---

## 2. Implement Silhouette Coefficient 
A good resource (with references) for clustering performance evaluation is scikit-learn's [documentation page](https://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation). It covers most of the methods. <br>
In this practice, we're going to implement the method 2.3.10.5. Silhouette Coefficient from the resource when ground truth labels are not known.

### 2.1 Obtain Clustering Result
We're going to run KMeans on Iris data, both algorithm and data are provided by Scikit-learn.

In [2]:
# Get clustering result from KMeans Alogorithm

X, y = datasets.load_iris(return_X_y=True)
kmeans_model = KMeans(n_clusters=3, random_state=1).fit(X)
labels = kmeans_model.labels_
labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2,
       2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0])

In [29]:
X[:2]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2]])

### 2.2 Get  Silhouette Score from Scikit-learn 

In [30]:
metrics.silhouette_score(X, labels, metric='euclidean')

0.5528190123564091

### 2.3 Implement your own Silhouette Score

We're using Iris data to valide the correctness of your code, While a toy data set as following could be used for debugging.

| f1 | f2 | Prediction|
|----|----|-----------|
| 1  | 1  | 1  |
| 2  | 2  | 1  |
| 3  | 3  | 0  |
| 4  | 4  | 0  |


### 2.3.1 Get Mean Distance between one sample and a group of samples
 

In [67]:
# Calculate the means distance between the given sample and a group of samples
def get_mean_dist(X, sample_idx, group_idx):    
    # parameter:  X               the data set
    # parameter:  sample_idx      index of the given sample
    # parameter:  group_idx       a list of indexes
    # return:     the mean distances
    # Your code goes here
#     X_points = [X[i] for i in group_idx] #use group_idx to get corresponding X rows   
    sample = X[sample_idx].reshape(4, 1)
#     return np.mean(np.linalg.norm(np.array(X_points) - sample.T, axis=1))
    return np.mean(pairwise_distances(X[group_idx], sample.T, metric='euclidean')) 
#     return np.mean(pairwise_distances(X_points, sample.T, metric='euclidean'))

In [68]:
# Validate your code, should print 0.4997..
get_mean_dist(X,0,range(50))

0.49978732149092736

### 2.3.2 Get `a` value
**a**: The mean distance('Euclidean') between a sample and all **other** points in the same class. <br>
Implement the calculation in the function *get_a(X,labels,sample_idx)*

In [69]:
def get_a(X, labels, sample_idx):
    # parameter:  X               the data set
    # parameter:  labeles         clustering result
    # parameter:  sample_idx      the given sample's index
    # return:     a value         the mean distances between the given sample and all samples in its cluster    
    # Your code goes here
    
    value = labels[sample_idx]        # get the value of the corresponding index (equivalent to sample_idx) in labels
    temp = np.where(labels == value)  # get indices of values that equals to sample_idx's value  
    group_idx = [i for i in temp[0]]  # convert from numpy array to a list
    
    if sample_idx in group_idx:
        group_idx.remove(sample_idx)  # remove the sample from the group  
    return get_mean_dist(X, sample_idx, group_idx)    

In [70]:
# Validate your code, should print 0.5099..
get_a(X, labels, 0)

0.5099870627458443

### 2.3.3 Get `b` value
**b**: The mean distance between a sample and all other points in the next nearest cluster. <br>
- determine the next nearest by calculating mean distance between the sample and clusters
- Implement the calculation in the function *get_b(X,labels,sample_idx)*

In [71]:
def get_b(X, labels, sample_idx):
    
    # parameter:  X               the data set
    # parameter:  labeles         clustering result
    # parameter:  sample_idx      the given sample's index
    # return:     b value         the smallest distances between the given sample and all samples in other clusters
    
    # Your code goes here
    val = labels[sample_idx]  # get the value of the corresponding index (equivalent to sample_idx) in labels
    alist = np.unique(labels) # gather all the unique values in label set into alist
    
    if val in alist:          # check whether the value of sample_idx is present in alist
        others = np.delete(alist, np.where(alist == val)) # if present, remove it from alist, remaining put in others  
    
    shortest = []
    for i in others:          # loop thr list containing unique values in the label set (except that of the sample_idx)        
        temp = np.where(labels == i)      # get indices of values that equals to sample_idx's value  
        group_idx = [i for i in temp[0]]  # convert from numpy array to a list
        if sample_idx in group_idx:
            group_idx.remove(sample_idx)  # remove the sample from the group  
        shortest.append(get_mean_dist(X, sample_idx, group_idx)) #append the iteration outcome to list shortest
  
    return min(shortest)

In [72]:
# Validate your code, should print 3.46..
get_b(X, labels, 0)

3.4682394501345906

### 2.3.3 Silhouette Score
The Silhouette Coefficient s for a single sample is then given as:
$$
s=\frac{b-a}{max(a,b)}
$$
The Silhouette Coefficient for a set of samples is given as the mean of the Silhouette Coefficient for each sample.<br>
Implete the calculation in function *get_s* that return the Silhouette Coefficient for a set of samples.

In [73]:
def get_s(X, labels):
    
    # parameter:  X               the data set
    # parameter:  labeles         clustering result
    # return:     Silhouette Score
    # Your code goes here
    total = 0
    for i in range(len(labels)):
         total += ((get_b(X, labels, i) - get_a(X, labels, i)) / max([get_b(X, labels, i), get_a(X, labels, i)]))
    return total / len(labels)

In [74]:
# Validate your code, should print 0.5528..
print(get_s(X,labels))

0.552819012356409


## 3 Summary

The best Silhouette score is 1 and the worst value is -1. Values near 0 indicate overlapping clusters. Negative values generally indicate that a sample has been assigned to the wrong cluster, as a different cluster is more similar.